In [ ]:
import numpy as np
import scipy 

### Question 3
Fit a series of successive polynomial models to the data: i.e. first fit
a constant, then fit a linear model, then fit a quadratic model, then a cubic model, etc. What
is the degree of the first degree of the first model that you would accept, at the 1% confidence
level? What is the degree of the first model that you would accept at the 1% confidence level? 

In [3]:
xi = np.array([i for i in range(1, 21)])
print("xi: ", xi)

Y = np.array([141, 144, 148, 150, 158, 161, 166, 170, 175, 181, 189, 194, 196, 206, 218, 229, 234, 242, 247, 257])
print("Y: ", Y)

xi:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Y:  [141 144 148 150 158 161 166 170 175 181 189 194 196 206 218 229 234 242
 247 257]


In [4]:
def orthog_poly(n, xi):
    # Define the first two orthogonal polynomials
    psi = []
    phi = []
    psi.append(lambda t: np.ones_like(t))
    psi.append(lambda t: t - np.mean(t))

    for i in range(2, n+1):
        a = np.sum(xi*(psi[i-1](xi))**2)/np.sum((psi[i-1](xi))**2)
        b = np.sum(psi[i-1](xi)**2)/np.sum(psi[i-2](xi)**2)
        
        psi.append(lambda t, i=i, a=a, b=b: (t - a)*psi[i-1](t) - b*psi[i-2](t))
    
    for i in range(n + 1):
        phi.append(lambda t, i=i: psi[i](t)/np.sqrt(np.sum(psi[i](xi)**2)))    
    
    return phi
   

In [19]:
p_value = .01

# Fit a series of successive polynomial models to the data
for poly_size in range(1, len(Y)):
    orthog_poly_basis = orthog_poly(poly_size, xi)
    X_orth =np.array([q(xi) for q in orthog_poly(poly_size, xi)]).T
    beta_hat_orth = X_orth.T @ Y
    
    RSS = np.sum((Y - X_orth @ beta_hat_orth)**2)
    #print("RSS: ", RSS)
    #print("beta_hat_orth: ", beta_hat_orth)
    
    # under null hypothesis that the leading coefficient is zero
    A = np.zeros((1, poly_size + 1))
    A[:, -1] = 1

    c = 0
    beta_h0 = beta_hat_orth + A.T@ np.linalg.inv(A @ A.T) @ (c -  A @ beta_hat_orth)
    # print("beta_h0: ", beta_h0)
    # RSS_h0 = np.sum((Y -  X_orth @ beta_h0 )**2)
    RSS_h0 = (Y -  X_orth @ beta_h0 ).T @ (Y -  X_orth @ beta_h0 )
    #print("RSS_h0: ", RSS_h0)
    
    # calculate f stat
    f_stat_crit = np.abs(RSS_h0 - RSS)/RSS * (len(Y) - poly_size)

    #print("diff: ", RSS_h0 - RSS)
    #print("f_stat: ", f_stat_crit)
    print("poly_size: ", poly_size)
    
    p_compare =  1 - scipy.stats.f.cdf(f_stat_crit, 1, len(Y) - poly_size)
    print("p_value: ", p_compare)
    if p_value > p_compare:
        print("Reject null hypothesis\n")
    else:
        print("Fail to reject null hypothesis\n")
        

poly_size:  1
p_value:  1.1102230246251565e-16
Reject null hypothesis

poly_size:  2
p_value:  1.1007069034008055e-08
Reject null hypothesis

poly_size:  3
p_value:  0.6458808944859353
Fail to reject null hypothesis

poly_size:  4
p_value:  0.07515056187201463
Fail to reject null hypothesis

poly_size:  5
p_value:  0.1597879250036146
Fail to reject null hypothesis

poly_size:  6
p_value:  0.36960429118117355
Fail to reject null hypothesis

poly_size:  7
p_value:  0.08863805302818262
Fail to reject null hypothesis

poly_size:  8
p_value:  0.12385260108166918
Fail to reject null hypothesis

poly_size:  9
p_value:  0.6383626921996918
Fail to reject null hypothesis

poly_size:  10
p_value:  0.07778119221002755
Fail to reject null hypothesis

poly_size:  11
p_value:  0.31868242856920914
Fail to reject null hypothesis

poly_size:  12
p_value:  0.05072620721736554
Fail to reject null hypothesis

poly_size:  13
p_value:  0.12413741051150229
Fail to reject null hypothesis

poly_size:  14
p_valu

The degree of the first model I would accept at the 1% confidence level is 2. See above. 

This is an odd question because it asks whether the improvement over the previous model is statistically significant. So, obviously a large improvement would be made from the linear model to the quadratic. It is odd that this also happens at degree = 19. 